<a href="https://colab.research.google.com/github/pinakm9/DeepRFM/blob/main/colab/DeepRFM_search_beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# run this cell to download data and necessary modules
import os, shutil
repo = 'DeepRFM'
if os.path.isdir(repo):
  shutil.rmtree(repo)
!git clone https://github.com/pinakm9/DeepRFM.git
# add modules folder to Python's search path
import os, sys
from pathlib import Path
script_dir = Path(os.path.dirname(os.path.abspath('')))
sys.path.insert(0, '/content/' + repo + '/modules')

import numpy as np
import utility as ut
import matplotlib.pyplot as plt
import pandas as pd
import rfm, skipRFM, deepSkip, localSkip_2_2, localSkip_8_1, localDeepSkip_2_2, localDeepSkip_8_1
import torch
from torch import nn
import l63, l96, ks
import config
from google.colab import drive
drive.mount('/content/gdrive', force_remount=False)

Cloning into 'DeepRFM'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 95 (delta 50), reused 77 (delta 32), pack-reused 0 (from 0)
Receiving objects: 100% (95/95), 211.45 KiB | 2.68 MiB/s, done.
Resolving deltas: 100% (50/50), done.
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
config_id = 'config_1'
device = "cuda" if torch.cuda.is_available() else "cpu"
L0, L1, placeholder_beta = 0.4, 3.5, 7e-8
data_folder = f"/content/{repo}/data"

dynamical_systems = ['L63']
ds_modules = {'L63': l63, 'L96': l96, 'KS': ks}
arch_modules = {'RFM': rfm, 'SkipRFM': skipRFM, 'DeepSkip': deepSkip, 'LocalSkip_2_2': localSkip_2_2, 'LocalSkip_8_1': localSkip_8_1,\
                'LocalDeepSkip_2_2': localDeepSkip_2_2, 'LocalDeepSkip_8_1': localDeepSkip_8_1}
for dynamical_system in dynamical_systems:
    print(f"Working on {dynamical_system}: generating training and testing data ...")
    # generate training and testing data
    setup = getattr(config, config_id)(dynamical_system)
    train, test = ds_modules[dynamical_system].gen_data(**setup['data_gen'])
    train, test = torch.tensor(train, device=device), torch.tensor(test, device=device)
    # setup architecture and test beta
    for architecture in setup['beta_arch']:
        print(f"Working on {dynamical_system}-{architecture} ...")
        D_r_list, B_list = list(zip(*setup['beta_arch'][architecture]))
        drf_args = [D_r_list[0], B_list[0], L0, L1, train, placeholder_beta, architecture,\
                    f"{data_folder}/{dynamical_system}/{config_id}/{architecture}/beta", False]
        beta_tester = rfm.BetaTester(arch_modules[architecture].DeepRF, D_r_list, B_list, [train]*len(D_r_list), test, *drf_args)
        beta_tester.search_beta(**{**setup['beta'], **setup['prediction_time']})

Working on L63: generating training and testing data ...
Time taken by gen_data is 7.1660 seconds
Working on L63-RFM ...
Estimated time to find optimal beta for (D_r, B) = (512, 1) is 0.04 hours
Estimated time to find optimal beta for (D_r, B) = (1024, 1) is 0.04 hours
Estimated time to find optimal beta for (D_r, B) = (2048, 1) is 0.05 hours
Estimated time to find optimal beta for (D_r, B) = (4096, 1) is 0.06 hours
Estimated time to find optimal beta for (D_r, B) = (8192, 1) is 0.09 hours
Estimated time to find optimal beta for (D_r, B) = (16384, 1) is 0.26 hours
Estimated time to find optimal beta for all (D_r, B) for current architecture is 0.54 hours
Experiments for (D_r, B, beta) = (512, 1, 1.00E-11) took 1.58E+00s
Experiments for (D_r, B, beta) = (512, 1, 1.36E-11) took 1.71E+00s
Experiments for (D_r, B, beta) = (512, 1, 1.72E-11) took 2.08E+00s
Experiments for (D_r, B, beta) = (512, 1, 2.08E-11) took 2.09E+00s
Experiments for (D_r, B, beta) = (512, 1, 2.44E-11) took 1.87E+00s
Ex

In [ ]:
!zip -r /content/data.zip /content/DeepRFM/data
!cp /content/data.zip "/content/gdrive/My Drive/Pinak/DeepRFM"